<center>
<img src='https://camo.githubusercontent.com/ad20aaaa48264581c46aa126919dbb72060ebd98cffe015523549b4974843ae5/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f46556c79616e6b696e2f725f70726f626162696c6974792f6d61737465722f656e645f73656d696e6172732f73656d5f362f73747564656e745f746561636865725f646973742e706e67' align='center' width="800x">
</center>

# Андан на экономе

## Семинар 11:  гипотетический семинар

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels as sts 

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from tqdm.notebook import tqdm

* Возможности `scipy` для проверки гипотез смотрите [в разделе Statistical tests](https://docs.scipy.org/doc/scipy/reference/stats.html)
* Возможности `statsmodels`  [в модуле stats](https://www.statsmodels.org/0.6.1/stats.html)

> Если используете встроенные функции, внимательно читайте документацию и проверяйте как именно расчитываются параметры.

## Упражнение 1 (котики и печеньки)

В этой задачке мы посмотрим на данные одной мобильной игрушки под названием "Cookie Cats", разработанной компанией <a href="http://tactile.dk">Tactile Entertainment</a>. Это стандартный match-three пазл, где игрокам нужно соединять элементы одинаковых цветов, чтобы очистить поле и выиграть на уровне. И еще там есть поющие коты.

<p><a href="https://youtu.be/GaP5f0jVTWE"><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cookie_cats_video.jpeg" style="width: 500px"></a></p>

По мере того, как игроки проходят уровни, они встречают ворота-блокеры, которые вынуждают их либо ждать длительное время, либо совершить внутриигровую покупку, чтобы продолжить играть. Помимо того, что эти ворота приносят разработчикам деньги от покупок, они еще и вынуждают игроков делать перерыв, в надежде на то, что те вернутся с новыми силами. 

<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cc_gates.png" alt=""></p>

Но возникает вопрос - когда и где ставить эти ворота? Изначально первые ворота стояли на 30-м уровне игры, однако в этом ноутбуке мы будем анализировать АБ-тест, в котором разработчики передвинули ворота на 40-й уровень. В частности мы будем смотреть влияние этого изменения на такой важный показатель как "retention" или удержание игроков, который расчитывается как отношение уникальных игроков, зашедших в игру на 1-й, 2-й, ... N-й день после установки ко всем игрокам, установившим приложение в этот день. 

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/hse-econ-data-science/andan_2023/main/sem11_ab/cookie_cats.csv')
print(df.shape)
df.head()

<ul>
<li><code>userid</code> - уникальный номер, идентифицирующий каждого игрока.</li>
<li><code>version</code> - был ли пользователь отнесен к контрольной группе (<code>gate_30</code> - ворота на 30-м уровне) или к тестовой (<code>gate_40</code> - ворота на 40-м уровне).</li>
<li><code>sum_gamerounds</code> - число сессий, сыгранных игроком в течение первых 14 дней после установки игры.</li>
<li><code>retention_1</code> - вернулся ли игрок после <strong>первого</strong> дня с момента установки?</li>
<li><code>retention_7</code> - вернулся ли игрок после <strong>седьмого</strong> дня с момента установки?</li>
</ul>

Когда игрок устанавливает игру, он/она случайным образом относятся либо к группе gate_30, либо gate_40. На всякий случай, давайте проверим, действительно ли их примерно поровну в каждой из групп.

In [ ]:
df.groupby('version').count()

<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/mr_waffles_smiling.png" style="width:200px; float:left"> </p>
<p>Похоже, что игроков действительно примерно поровну в каждой из групп, отлично!</p>
<p>Фокус нашего анализа будет сосредоточен на удержании игроков (retention), но ради интереса давайте построим распределение числа игровых сессий, сыгранных игроками в течение их первой недели жизни в игре.</p>

In [ ]:
# Считаем число игроков, оставшееся в игре для каждого раунда
plot_df = df.groupby('sum_gamerounds')['userid'].count()

# Распределение людей, сыгравших от 0 до 100 раундов
ax = plot_df.head(100).plot()
ax.set_xlabel("sum_gamerounds")
ax.set_ylabel("number of players");

<p>На графике вверху мы видим, что некоторые игроки установили игру, но даже ни разу не поиграли (0 сессий), многие игроки закончили лишь пару сессий за первую неделю, а некоторые действительно подсели и сыграли более 80 раз!</p>
<p>Конечно, мы хотим, чтобы игроки были заинтересованы в игрушке и возвращались в неё снова и снова. Обычная метрика, использующаяся в гейм-индустрии, чтобы измерить, насколько игрушка веселая и захватывающая, - это <em>удержание первого дня</em> (1-day retention): Процент игроков, которые вренулись и снова стали играть спустя 1 день после установки. Чем выше удержание первого дня, тем проще и дальше удерживать пользователей и строить себе большую базу фанатов.</p>
<p>В качестве первого шага, давайте посмотрим, как в целом выглядит 1-day retention.</p>

In [ ]:
df.retention_1.mean()

<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/belle_cookie.png" style="width:200px; float:right"> </p>
<p>Итак, немногим меьше половины всех игроков возвращяются к нам спустя один день после установки. Теперь, когда у нас есть это базовое значение (benchmark), давайте посмотрим, как отличается 1-day retention внутри наших тестовых групп.</p>

In [ ]:
df.groupby('version').retention_1.mean()

<p>Похоже, что у нас есть небольшое ухудшение в удержании первого дня, если мы двигаем ворота к сороковому уровню (44.2%) в сравнеии с контрольной группой, где ворота остались на 30-м уровне (44.8%). Вполне естественен вопрос, а значима ли эта разница? 

__а)__  Постройте примерный $95\%$-й доверительный интервал для каждой из двух долей.

$$
\left[\hat p - z_{crit} \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}} ; \quad  \hat p + z_{crit} \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}} \right]
$$

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__б)__ Постройте примерный $95\%$-й доверительный интервал для разницы долей.

$$
\left[\hat p_A - \hat p_B - z_{crit} \sqrt{\frac{\hat p_A \cdot (1 - \hat p_A)}{n_A} + \frac{\hat p_B \cdot (1 - \hat p_B)}{n_B}} ; \quad  \hat p_A - \hat p_B + z_{crit} \sqrt{\frac{\hat p_A \cdot (1 - \hat p_A)}{n_A} + \frac{\hat p_B \cdot (1 - \hat p_B)}{n_B}} \right]
$$

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

Какие выводы можно сделать на основе этих двух пунктов? Обратите внимание, что в пункте __а)__ хочется сделать выводы о влиянии ворот на удержание по пересечению доверительных интервалов. Это очень порочная практика. О ней мы поговорим подробнее в последнем упражнении этой тетрадки. 

__в)__ Проверьте гипотезу о том, что перенос ворот не оказывает никакого влияния на удержание первого дня.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__г)__ Проверьте гипотезу о том, что перенос ворот не оказывает никакого влияния на удержание седьмого дня.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__д)__ Предположим, что в нашу игру ежедневно играет 100 тысяч человек. Насколько существенный эффект окажет на бизнес перенос ворот на 40 уровень?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

<p>Итак, результаты бутстрапа говорят нам о том, что есть значительное доказательство превышения 7-day retentino в группе с воротами на 30-м уровне над группой с воротами на 40-м. Значит, если мы хотим держать ретеншн на высоком уровне и иметь большее число игроков, нам <strong>не нужно</strong> сдвигать ворота с 30-го на 40-й уровень.

<p>Так почему же ретеншн выше, когда ворота расположены раньше? Логично было бы ожидать обратной ситуации - чем позже ставится препятствие, тем дольше игроки будут взаимодействовать с игрой. Однако это не то, что мы видим в данных. Теория гедонистической адаптации может помочь с объяснением. Если вкратце, гедонистическая адаптация - это тенденция получать всё меньше и меньше удовольствия от деятельности, если она продолжается длительное время. Заставляя игроков сделать паузу, когда они достигают 30-го уровня, мы продлеваем им удовольствие от игры, в результате чего они хотят в неё возвращаться. И напротив, сдвигая ворота к 40-му уровню мы даем игрокам возможность наиграться и просто заскучать.</p>

<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cookie_yellow.png" style="width:100px; float:center"> </p>

## Упражнение 2 (ошибки 1 и 2 рода)

Коля Коперник задумал что-то вроде АБ-теста. Он хочет проверить известное утверждение о том, что бутерброд чаще падает маслом вниз. Николай использует асимптотический тест для долей

$$
z = \frac{\hat p - p_0}{\sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}}}
$$

и формулирует свои мысли в виде следующей гипотезы 

$$
\begin{aligned}
&H_0: \hspace{2mm} p = 0.5 \\
&H_1: \hspace{2mm} p > 0.5 
\end{aligned}
$$

Перед проведением АБ-теста обычно фиксируют уровень значимости, $\alpha$. 

- __Ошибка первого рода (уровень значимости)__ - вероятность отвергнуть гипотезу $H_0$, когда она верна. Попробуем с помощью симуляций вычислить её для проверки гипотезы о среднем.

- __Ошибка второго рода__ - вероятность не отвергнуть гипотезу $H_0$, когда она неверна.

__а)__  Пусть Николай зафикисировал $\alpha = 0.05$ и собрал $100$ наблюдений, а настоящее значение $p = 0.5$. 

С помощью симуляций  проверьте гипотезу Коперника $10^4$ раз и убедитесь, что доля ошибок действительно будет $5\%.$

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__б)__  Пусть Николай зафикисировал $\alpha = 0.05$ и собрал $100$ наблюдений, а настоящее значение $p = 0.6$. С помощью симуляций вычислите ошибку второго рода для нашего эксперимента. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__в)__ Пусть Николай собрал не $100$, а $200$ наблюдений. Какой будет ошибка второго рода в таком случае? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

Давайте зафиксируем $\alpha=0.05$, а $p=0.6$. Переберите число наблюдений от 10 до 1000 с шагом 50 и нарисуйте график зависимости ошибки второго рода от числа наблюдений.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

Объясните, почему нельзя нарисовать такой же график для ошибки первого рода? Что такое презумпция нулевой гипотезы? 

__г)__ Перебирите $\alpha$ в диапазоне $(0; 1)$ с шагом $0.1$. Найдите для каждого значения $\alpha$ значение ошибки второго рода $\beta$. Изобразите на графике получившуюся зависимость для $n = 100$ и $n = 200$. Объясните, что получилось.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__MDE (minimal detectable effect) -__ минимальная величина, которую мы хотим поймать нашим АБ-тестом. В случае симуляций выше это была величина $p_a - p_0 = 0.6 - 0.5 = 0.1$. Чем меньше $MDE$, тем сложнее различить две ситуации и тем больше наблюдений нам понадобится.

Ошибку первого рода и MDE обычно фиксируют перед проведением АБ-теста. Ошибку второго рода мы контролируем с помощью числа наблюдений. На самом деле, для критерия рассмотренного выше, можно вывести формулу для поиска числа необходимых наблюдений:

$$
n(\alpha, \beta, MDE)= \frac{\hat{p}_A \cdot (1 - \hat{p}_A) + \hat{p}_B \cdot (1 - \hat{p}_B)}{0.5 \cdot MDE^2 } \cdot (z_{1-\alpha} + z_{1-\beta})^2
$$

Вывод этой формулы можно найти, например, [в лекции Фила.](https://www.youtube.com/watch?v=LTlqyRI8cQ0&list=PLNKXA-74YGLjDOtDSZEFoy1yP-3AfiHUC&index=16) А ещё [прочитайте историю](https://absentis.livejournal.com/27153.html) про то, почему Коперник мазал бутеры маслом. 

На практике, перед запуском эксперимента обычно готовят табличку, в которой смотрят на то, сколько наблюдений понадобится для поиска эффекта разного размера при разных значениях ошибок первого и второго рода. Для разных критериев в интернете можно найти калькуляторы размера выборок.

In [ ]:
def get_n_with_p(MDE, alpha, beta, p1=0.010, p2=0.015):
    za = stats.norm().ppf(1 - alpha)
    zb = stats.norm().ppf(1 - beta)
    ans = 1/2 *(p1 * (1-p1) + p2 * (1-p2)) * ((za + zb) / MDE)**2
    return ans 

In [ ]:
MDE = 0.01 # Если мы хотим поймать маленькое изменение, надо очень много наблюдений

alphas = [0.001, 0.01, 0.02, 0.05, 0.1, 0.2]
betas = [0.001, 0.01, 0.02, 0.05, 0.1, 0.2]

Errors = [ ]
for a in alphas:
    cur_error = []
    for b in betas:
        e = get_n_with_p(MDE, a, b)
        cur_error.append(e)
    Errors.append(cur_error)

df_e = pd.DataFrame(Errors)
df_e.columns = alphas
df_e.index = betas
df_e

https://xkcd.com/1478/

<img src="https://imgs.xkcd.com/comics/p_values_2x.png" height="300" width="300">

## Упражнение 3 (повторное тестирование)

Жизнь исследователя Винни-Пуха прекрасна. Наблюдений много, гипотеза о равенстве математических ожиданий верна, а наблюдения нормальны.

А именно, величины $X_1$, ..., $X_{n}$ независимы и нормальны $N(0;1)$, величины $Y_1$, ..., $Y_n$ независимы и нормальны $N(0;2)$. И пусть $n_x=100$ и $n_y=200$.

Винни-Пух правда этого ничего не знает, потому что у него в голове опилки, и тестирует сначала гипотезу о равенстве дисперсий двух выборок на уровне значимости $5\%$.

__а)__ Проведите $10^6$ экспериментов, то есть $10^6$ раз попробуйте за Винни-Пуха проверить гипотезу. Хорошо бы $10^8$, но оттестируйте сначала свой код на $10^6$. 

Постройте гистограмму распределения тестовой статистики о равенстве дисперсий. В скольки процентах случаев гипотеза о равенстве дисперсий отвергнута? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__б)__ Если гипотеза о равенстве дисперсий не отвергается, то Винни использует $t$-статистику для проверки гипотезы о равенстве ожиданий при равенстве дисперсий. 

Для тех экспериментов, в которых гипотеза о равенстве дисперсий не отверглась, проверьте гипотезу о равенстве ожиданий на уровне значимости $5\%$. 

Постройте гистограмму распределения $t$-статистики. В скольки процентах случаев гипотеза о равенстве ожиданий отвергнута?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

__в)__ Словами объясните, почему нехорошо сначала тестировать на равенство дисперсий, а потом на равенство средних при равных дисперсиях, если на первом шаге гипотеза $H_0$ не отверглась. 

https://xkcd.com/882/

![](https://imgs.xkcd.com/comics/significant.png)

## Упражнение 4 (распределение p-value)

С помощью симуляций нарисуйте гистограмму для распределени p-value в случае верности нулевой гипотезы и в случае верности альтернативной гипотезы. Объясните, что означают такие распределения.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

## Упражнение 5 (пересечение Vs разность)

Иногда люди делают ужасное. Вместо того, чтобы проверить гипотезу о равенстве средних, они смотрят пересекаются ли доверительные интервалы. Так делать нельзя.

Дело в том, что для одинаковых ошибок первого рода, $\alpha$, ошибка второго рода, $\beta$, для процедуры, основанной на доверительных интервалах, окажется выше. Задание состоит в том, чтобы это увидеть.

### Процедура 1 (правильная):

1. Собираем выборки $X_1, \ldots, X_n$ и $Y_1, \ldots, Y_n$;
2. Находим значение статистики

$$
z_{obs} = \frac{\bar x - \bar y}{\sqrt{\frac{s_x^2}{n_x} + \frac{s_y^2}{n_y}}};
$$

3. Говорим, что по ЦПТ $z_{obs} \overset{asy}{\sim} N(0,1);$
4. Находим критическое значение $z_{1 - \frac{\alpha}{2}}$;
5. Если мы видим, что $|z_{obs}| <  z_{1 - \frac{\alpha}{2}}$, мы говорим, что гипотеза не отвергается. 



### Процедура 2 (мерзкая): 

1. Собираем выборки $X_1, \ldots, X_n$ и $Y_1, \ldots, Y_n$;
2. Находим $\bar x$ и $\bar y$;
3. Пользуясь ЦПТ и зная, что $\bar x \overset{asy}{\sim} N \left(\mu_1,\frac{s^2_x}{n_x} \right)$ и $\bar y \overset{asy}{\sim} N\left(\mu_2,\frac{s^2_y}{n_y}\right)$ строим для $\mu_1$ и $\mu_2$ доверительные интервалы;
4. Если доверительные интервалы пересеклись, говорим, что гипотеза не отвергается. 

Зафиксируйте количество наблюдений, MDE и уровень значимости. Убедитесь с помощью симуляций в том, что ошибка второго рода для второй процедуры будет больше, чем для первой. 

Из-за того что мы строим два доверительных интервала вместо одного, ошибка 1 рода выходит из-под контроля. Какой она оказывается на самом деле?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

Для этого упражнения можно получить формулы в явном виде. Попробуйте сделать это. Готовый вывод можно найти [в лекции Фила.](https://www.youtube.com/watch?v=6Bcjc_0N_jw&list=PLNKXA-74YGLjDOtDSZEFoy1yP-3AfiHUC&index=17&t=3s)